## The challenge:

In this competition you’ll predict what types of trees there are in an area based on various geographic features.

The competition datasets comes from a study conducted in four wilderness areas within the beautiful Roosevelt National Forest of northern Colorado. These areas represent forests with very little human disturbances – the existing forest cover types there are more a result of ecological processes rather than forest management practices.

The data is in raw form and contains categorical data such as wilderness areas and soil type.

## Import Packages

In [1]:
DATA_DIR = '/kaggle/input/learn-together'
DATA_DIR = 'data'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard

for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pybr/kaggle-fores

data/train.csv
data/test.csv
data/sample_submission.csv
data/sample_submission.csv.zip
data/input
data/test.csv.zip
data/train.csv.zip


/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/pybr/kaggle-forest-types/venv/lib/python3.5/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/i

In [3]:
print(tf.__version__)

1.14.0


In [4]:
def report(y_true, y_pred):
    print('Accuracy: %s' % accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

## Load Dataset

In [5]:
train_df=pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df=pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [6]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [7]:
test_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [8]:
print("shape training csv: %s" % str(train_df.shape)) 
print("shape test csv: %s" % str(test_df.shape)) 

shape training csv: (15120, 56)
shape test csv: (565892, 55)


## Delete Ids
**Let's delete the Id column in the training set but store it for the test set before deleting**

In [9]:
train_df = train_df.drop(["Id"], axis = 1)

test_ids = test_df["Id"]
test_df = test_df.drop(["Id"], axis = 1)

In [10]:
def new_features(df):
    df = df.copy()
    df['Hydro_Elevation_diff'] = df[['Elevation',
                                   'Vertical_Distance_To_Hydrology']
                                  ].diff(axis='columns').iloc[:, [1]]

    df['Hydro_Euclidean'] = np.sqrt(df['Horizontal_Distance_To_Hydrology']**2 +
                                   df['Vertical_Distance_To_Hydrology']**2)

    df['Hydro_Fire_sum'] = df[['Horizontal_Distance_To_Hydrology',
                             'Horizontal_Distance_To_Fire_Points']
                            ].sum(axis='columns')

    df['Hydro_Fire_diff'] = df[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Fire_Points']
                             ].diff(axis='columns').iloc[:, [1]].abs()

    df['Hydro_Road_sum'] = df[['Horizontal_Distance_To_Hydrology',
                             'Horizontal_Distance_To_Roadways']
                            ].sum(axis='columns')

    df['Hydro_Road_diff'] = df[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Roadways']
                             ].diff(axis='columns').iloc[:, [1]].abs()

    df['Road_Fire_sum'] = df[['Horizontal_Distance_To_Roadways',
                            'Horizontal_Distance_To_Fire_Points']
                           ].sum(axis='columns')

    df['Road_Fire_diff'] = df[['Horizontal_Distance_To_Roadways',
                             'Horizontal_Distance_To_Fire_Points']
                            ].diff(axis='columns').iloc[:, [1]].abs()
    df['Stoneyness'] = sum(i * df['Soil_Type{}'.format(i)] for i in range(1, 41))

    # For all 40 Soil_Types, 1=rubbly, 2=stony, 3=very stony, 4=extremely stony, 0=?
    stoneyness = [4, 3, 1, 1, 1, 2, 0, 0, 3, 1, 
                  1, 2, 1, 0, 0, 0, 0, 3, 0, 0, 
                  0, 4, 0, 4, 4, 3, 4, 4, 4, 4, 
                  4, 4, 4, 4, 1, 4, 4, 4, 4, 4]

    # Replace Soil_Type number with "stoneyness" value
    df['Stoneyness'] = df['Stoneyness'].replace(range(1, 41), stoneyness)
    return df


In [11]:
train_df = new_features(train_df)
test_df = new_features(test_df)

In [12]:
train_df.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [13]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ss = StandardScaler()
mm = MinMaxScaler()
numerical_df = train_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points',
       'Hydro_Elevation_diff',
       'Hydro_Euclidean', 'Hydro_Fire_sum', 'Hydro_Fire_diff',
       'Hydro_Road_sum', 'Hydro_Road_diff', 'Road_Fire_sum', 'Road_Fire_diff',
       'Stoneyness']]

numerical_df_test = test_df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points',
       'Hydro_Elevation_diff',
       'Hydro_Euclidean', 'Hydro_Fire_sum', 'Hydro_Fire_diff',
       'Hydro_Road_sum', 'Hydro_Road_diff', 'Road_Fire_sum', 'Road_Fire_diff',
       'Stoneyness']]



In [14]:
ss.fit(pd.concat([numerical_df, numerical_df_test]))
X_ss = ss.transform(numerical_df)
X_ss_test = ss.transform(numerical_df_test)

In [15]:
def categorical_features(train_df, test_df):
    size = len(train_df)
    size_test = len(test_df)
    columns_to_keep = []
    for column in train_df.columns:
        if 'Soil' in column or 'Wilderness' in column:
            frequency = train_df[column].sum() / size
            if frequency >= 0.01:
                columns_to_keep.append(column)
    print('Columns keeped %s' % columns_to_keep)
    return np.array(train_df[columns_to_keep]), np.array(test_df[columns_to_keep])

In [16]:
categorical, categorical_test = categorical_features(train_df, test_df)

Columns keeped ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type17', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']


In [17]:
categorical[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0]])

In [18]:
X = np.hstack((X_ss, categorical))

In [19]:
X_test = np.hstack((X_ss_test, categorical_test))

In [20]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()
y = binarizer.fit_transform(train_df['Cover_Type'])

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [22]:
def model1():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [23]:
def model2():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [24]:
def model3():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(7, activation='softmax'))
    return model


In [25]:
def compile_and_train(model, epochs=50, verbose=0):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)
    print(model.metrics_names)
    print(model.evaluate(X_train, y_train))
    print(model.evaluate(X_val, y_val))

In [26]:
for model in [model1(), model2(), model3()]:
    compile_and_train(model)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
['loss', 'acc']
13608/13608 [==============================] - 1s 50us/sample - loss: 0.5104 - acc: 0.7906
[0.5103766819016503, 0.79056436]
1512/1512 [==============================] - 0s 20us/sample - loss: 0.5358 - acc: 0.7758
[0.5358185714514798, 0.7757937]
['loss', 'acc']
13608/13608 [==============================] - 0s 24us/sample - loss: 0.4505 - acc: 0.8134
[0.4504844614647894, 0.81341857]
1512/1512 [==============================] - 0s 22us/sample - loss: 0.4850 - acc: 0.8056
[0.4850093452703385, 0.8055556]
['loss', 'acc']
13608/13608 [==============================] - 0s 26us/sample - loss: 0.5131 - acc: 0.7895
[0.5131047436729871, 0.7894621]
1512/1512 [==============================] - 0s 23us/sample - loss: 0.5410 - acc: 0.7778
[0.5410123062196863, 0.7777778]


## Final training

### Callbacks

In [27]:
tensorboard_logs = TensorBoard(log_dir='./logs', histogram_freq=1,
                               write_graph=True, write_images=True,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join('./keras_models', 
                                        'modelweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_acc', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=60, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=40,
                              verbose=1, min_delta=1e-2, mode='min')

### Model

### Training

In [28]:
model = model2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=500,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          batch_size=378)


Train on 13608 samples, validate on 1512 samples
Epoch 1/500
13608/13608 [==============================] - 2s 154us/sample - loss: 1.7558 - acc: 0.2863 - val_loss: 1.3595 - val_acc: 0.4775
Epoch 2/500
13608/13608 [==============================] - 0s 7us/sample - loss: 1.3263 - acc: 0.4383 - val_loss: 1.0148 - val_acc: 0.5813
Epoch 3/500
13608/13608 [==============================] - 0s 7us/sample - loss: 1.1185 - acc: 0.5226 - val_loss: 0.8633 - val_acc: 0.6515
Epoch 4/500
13608/13608 [==============================] - 0s 7us/sample - loss: 1.0089 - acc: 0.5658 - val_loss: 0.7928 - val_acc: 0.6852
Epoch 5/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.9411 - acc: 0.5973 - val_loss: 0.7465 - val_acc: 0.6958
Epoch 6/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.9039 - acc: 0.6197 - val_loss: 0.7223 - val_acc: 0.7050
Epoch 7/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.8656 - acc: 0.6368 - val_loss: 0

13608/13608 [==============================] - 0s 6us/sample - loss: 0.5913 - acc: 0.7525 - val_loss: 0.5302 - val_acc: 0.7857
Epoch 56/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5961 - acc: 0.7530 - val_loss: 0.5339 - val_acc: 0.7824
Epoch 57/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5887 - acc: 0.7579 - val_loss: 0.5280 - val_acc: 0.7870
Epoch 58/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5890 - acc: 0.7572 - val_loss: 0.5237 - val_acc: 0.7831
Epoch 59/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5916 - acc: 0.7551 - val_loss: 0.5285 - val_acc: 0.7817
Epoch 60/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5909 - acc: 0.7585 - val_loss: 0.5301 - val_acc: 0.7870
Epoch 61/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5864 - acc: 0.7574 - val_loss: 0.5236 - val_acc: 0.7817
Epoch 62/500
13608/13608 [=======

13608/13608 [==============================] - 0s 7us/sample - loss: 0.5343 - acc: 0.7812 - val_loss: 0.4750 - val_acc: 0.8075
Epoch 114/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.5321 - acc: 0.7853 - val_loss: 0.4863 - val_acc: 0.8056
Epoch 115/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5351 - acc: 0.7828 - val_loss: 0.4774 - val_acc: 0.8075
Epoch 116/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5326 - acc: 0.7769 - val_loss: 0.4877 - val_acc: 0.8069
Epoch 117/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.5308 - acc: 0.7858 - val_loss: 0.4760 - val_acc: 0.8181
Epoch 118/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5279 - acc: 0.7822 - val_loss: 0.4780 - val_acc: 0.8122
Epoch 119/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5304 - acc: 0.7847 - val_loss: 0.4751 - val_acc: 0.8056
Epoch 120/500
13608/13608 [

13608/13608 [==============================] - 0s 7us/sample - loss: 0.5001 - acc: 0.7962 - val_loss: 0.4541 - val_acc: 0.8261
Epoch 172/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.5097 - acc: 0.7956 - val_loss: 0.4540 - val_acc: 0.8194
Epoch 173/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5047 - acc: 0.7942 - val_loss: 0.4537 - val_acc: 0.8221
Epoch 174/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4995 - acc: 0.8006 - val_loss: 0.4549 - val_acc: 0.8274
Epoch 175/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5031 - acc: 0.7956 - val_loss: 0.4525 - val_acc: 0.8254
Epoch 176/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.5022 - acc: 0.7952 - val_loss: 0.4506 - val_acc: 0.8234
Epoch 177/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.5035 - acc: 0.7947 - val_loss: 0.4480 - val_acc: 0.8280
Epoch 178/500
13608/13608 [

13608/13608 [==============================] - 0s 7us/sample - loss: 0.4795 - acc: 0.8066 - val_loss: 0.4361 - val_acc: 0.8294
Epoch 229/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4785 - acc: 0.8044 - val_loss: 0.4372 - val_acc: 0.8221
Epoch 230/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4736 - acc: 0.8070 - val_loss: 0.4385 - val_acc: 0.8208
Epoch 231/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4826 - acc: 0.8049 - val_loss: 0.4356 - val_acc: 0.8320
Epoch 232/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4791 - acc: 0.8084 - val_loss: 0.4347 - val_acc: 0.8327
Epoch 233/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4800 - acc: 0.8070 - val_loss: 0.4335 - val_acc: 0.8300
Epoch 234/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4839 - acc: 0.8040 - val_loss: 0.4399 - val_acc: 0.8261
Epoch 235/500
13608/13608 [

13608/13608 [==============================] - 0s 7us/sample - loss: 0.4675 - acc: 0.8123 - val_loss: 0.4320 - val_acc: 0.8274
Epoch 286/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4704 - acc: 0.8151 - val_loss: 0.4350 - val_acc: 0.8254
Epoch 287/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4619 - acc: 0.8161 - val_loss: 0.4392 - val_acc: 0.8267
Epoch 288/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4615 - acc: 0.8120 - val_loss: 0.4371 - val_acc: 0.8228
Epoch 289/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4680 - acc: 0.8131 - val_loss: 0.4348 - val_acc: 0.8254
Epoch 290/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4705 - acc: 0.8067 - val_loss: 0.4294 - val_acc: 0.8313
Epoch 291/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4623 - acc: 0.8160 - val_loss: 0.4296 - val_acc: 0.8320
Epoch 292/500
13608/13608 [

Epoch 341/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4568 - acc: 0.8155 - val_loss: 0.4241 - val_acc: 0.8366
Epoch 342/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4568 - acc: 0.8170 - val_loss: 0.4270 - val_acc: 0.8300
Epoch 343/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4570 - acc: 0.8172 - val_loss: 0.4276 - val_acc: 0.8280
Epoch 344/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4581 - acc: 0.8154 - val_loss: 0.4268 - val_acc: 0.8347
Epoch 345/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4540 - acc: 0.8172 - val_loss: 0.4245 - val_acc: 0.8360
Epoch 346/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4519 - acc: 0.8201 - val_loss: 0.4227 - val_acc: 0.8294
Epoch 347/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4601 - acc: 0.8153 - val_loss: 0.4295 - val_acc: 0.8294
Epoch 348/500

Epoch 398/500
13608/13608 [==============================] - 0s 6us/sample - loss: 0.4495 - acc: 0.8192 - val_loss: 0.4207 - val_acc: 0.8327
Epoch 399/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4422 - acc: 0.8222 - val_loss: 0.4186 - val_acc: 0.8333
Epoch 400/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4528 - acc: 0.8191 - val_loss: 0.4257 - val_acc: 0.8274
Epoch 401/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4513 - acc: 0.8180 - val_loss: 0.4208 - val_acc: 0.8327
Epoch 402/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4521 - acc: 0.8177 - val_loss: 0.4225 - val_acc: 0.8320
Epoch 403/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4588 - acc: 0.8166 - val_loss: 0.4197 - val_acc: 0.8327
Epoch 404/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4458 - acc: 0.8216 - val_loss: 0.4232 - val_acc: 0.8313
Epoch 405/500

Epoch 455/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4456 - acc: 0.8244 - val_loss: 0.4205 - val_acc: 0.8320
Epoch 456/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4442 - acc: 0.8211 - val_loss: 0.4234 - val_acc: 0.8320
Epoch 457/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4451 - acc: 0.8206 - val_loss: 0.4196 - val_acc: 0.8353
Epoch 458/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4420 - acc: 0.8213 - val_loss: 0.4226 - val_acc: 0.8327
Epoch 459/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4414 - acc: 0.8222 - val_loss: 0.4184 - val_acc: 0.8340
Epoch 460/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4431 - acc: 0.8250 - val_loss: 0.4207 - val_acc: 0.8313
Epoch 461/500
13608/13608 [==============================] - 0s 7us/sample - loss: 0.4430 - acc: 0.8229 - val_loss: 0.4161 - val_acc: 0.8366
Epoch 462/500

In [29]:
y_pred = model.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.833994708994709
              precision    recall  f1-score   support

           1       0.71      0.74      0.72       196
           2       0.74      0.62      0.68       219
           3       0.85      0.69      0.76       228
           4       0.91      0.95      0.93       221
           5       0.87      0.96      0.91       201
           6       0.76      0.91      0.83       216
           7       0.97      0.97      0.97       231

    accuracy                           0.83      1512
   macro avg       0.83      0.83      0.83      1512
weighted avg       0.83      0.83      0.83      1512

[[145  41   1   0   1   1   7]
 [ 53 136   2   0  23   5   0]
 [  0   2 158  17   2  49   0]
 [  0   0   7 210   0   4   0]
 [  0   2   5   0 192   2   0]
 [  0   1  13   4   2 196   0]
 [  6   1   0   0   0   0 224]]


In [30]:
model = model2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=500,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          batch_size=378)

Train on 15120 samples, validate on 1512 samples
Epoch 1/500
15120/15120 [==============================] - 1s 38us/sample - loss: 1.7366 - acc: 0.2893 - val_loss: 1.3231 - val_acc: 0.4603
Epoch 2/500
15120/15120 [==============================] - 0s 7us/sample - loss: 1.3190 - acc: 0.4415 - val_loss: 0.9881 - val_acc: 0.6171
Epoch 3/500
15120/15120 [==============================] - 0s 7us/sample - loss: 1.1187 - acc: 0.5259 - val_loss: 0.8455 - val_acc: 0.6766
Epoch 4/500
15120/15120 [==============================] - 0s 7us/sample - loss: 1.0021 - acc: 0.5692 - val_loss: 0.7732 - val_acc: 0.6951
Epoch 5/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9301 - acc: 0.6071 - val_loss: 0.7361 - val_acc: 0.7090
Epoch 6/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8906 - acc: 0.6272 - val_loss: 0.7115 - val_acc: 0.7222
Epoch 7/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8601 - acc: 0.6382 - val_loss: 0.

15120/15120 [==============================] - 0s 7us/sample - loss: 0.5918 - acc: 0.7581 - val_loss: 0.4788 - val_acc: 0.7937
Epoch 60/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5866 - acc: 0.7581 - val_loss: 0.4800 - val_acc: 0.7983
Epoch 61/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5862 - acc: 0.7585 - val_loss: 0.4747 - val_acc: 0.7989
Epoch 62/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5795 - acc: 0.7625 - val_loss: 0.4763 - val_acc: 0.8102
Epoch 63/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5795 - acc: 0.7624 - val_loss: 0.4684 - val_acc: 0.8102
Epoch 64/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5718 - acc: 0.7665 - val_loss: 0.4694 - val_acc: 0.8142
Epoch 65/500
15120/15120 [==============================] - 0s 8us/sample - loss: 0.5788 - acc: 0.7663 - val_loss: 0.4705 - val_acc: 0.8042
Epoch 66/500
15120/15120 [=======

15120/15120 [==============================] - 0s 7us/sample - loss: 0.5311 - acc: 0.7824 - val_loss: 0.4244 - val_acc: 0.8247
Epoch 118/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5289 - acc: 0.7803 - val_loss: 0.4163 - val_acc: 0.8313
Epoch 119/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5194 - acc: 0.7880 - val_loss: 0.4170 - val_acc: 0.8247
Epoch 120/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5305 - acc: 0.7845 - val_loss: 0.4157 - val_acc: 0.8313
Epoch 121/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5261 - acc: 0.7839 - val_loss: 0.4185 - val_acc: 0.8261
Epoch 122/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5314 - acc: 0.7813 - val_loss: 0.4245 - val_acc: 0.8313
Epoch 123/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5222 - acc: 0.7870 - val_loss: 0.4168 - val_acc: 0.8254
Epoch 124/500
15120/15120 [

15120/15120 [==============================] - 0s 7us/sample - loss: 0.5068 - acc: 0.7935 - val_loss: 0.4022 - val_acc: 0.8366
Epoch 176/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5047 - acc: 0.7933 - val_loss: 0.3958 - val_acc: 0.8393
Epoch 177/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4957 - acc: 0.7994 - val_loss: 0.3902 - val_acc: 0.8492
Epoch 178/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5014 - acc: 0.7964 - val_loss: 0.3981 - val_acc: 0.8406
Epoch 179/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4968 - acc: 0.7992 - val_loss: 0.3905 - val_acc: 0.8380
Epoch 180/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4974 - acc: 0.7975 - val_loss: 0.3908 - val_acc: 0.8386
Epoch 181/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4964 - acc: 0.7967 - val_loss: 0.3876 - val_acc: 0.8433
Epoch 182/500
15120/15120 [

15120/15120 [==============================] - 0s 7us/sample - loss: 0.4799 - acc: 0.8054 - val_loss: 0.3813 - val_acc: 0.8373
Epoch 234/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4829 - acc: 0.8050 - val_loss: 0.3794 - val_acc: 0.8499
Epoch 235/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4774 - acc: 0.8065 - val_loss: 0.3778 - val_acc: 0.8452
Epoch 236/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4828 - acc: 0.8035 - val_loss: 0.3766 - val_acc: 0.8512
Epoch 237/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4875 - acc: 0.8012 - val_loss: 0.3762 - val_acc: 0.8532
Epoch 238/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4869 - acc: 0.8022 - val_loss: 0.3743 - val_acc: 0.8512
Epoch 239/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4767 - acc: 0.8053 - val_loss: 0.3733 - val_acc: 0.8499
Epoch 240/500
15120/15120 [

15120/15120 [==============================] - 0s 6us/sample - loss: 0.4715 - acc: 0.8102 - val_loss: 0.3614 - val_acc: 0.8525
Epoch 292/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4714 - acc: 0.8093 - val_loss: 0.3649 - val_acc: 0.8558
Epoch 293/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4723 - acc: 0.8113 - val_loss: 0.3576 - val_acc: 0.8565
Epoch 294/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4749 - acc: 0.8088 - val_loss: 0.3656 - val_acc: 0.8558
Epoch 295/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4728 - acc: 0.8068 - val_loss: 0.3670 - val_acc: 0.8505
Epoch 296/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4733 - acc: 0.8072 - val_loss: 0.3584 - val_acc: 0.8538
Epoch 297/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4689 - acc: 0.8115 - val_loss: 0.3523 - val_acc: 0.8591
Epoch 298/500
15120/15120 [

Epoch 349/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4662 - acc: 0.8105 - val_loss: 0.3474 - val_acc: 0.8552
Epoch 350/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4618 - acc: 0.8149 - val_loss: 0.3495 - val_acc: 0.8618
Epoch 351/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4622 - acc: 0.8140 - val_loss: 0.3473 - val_acc: 0.8618
Epoch 352/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4643 - acc: 0.8122 - val_loss: 0.3441 - val_acc: 0.8677
Epoch 353/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4614 - acc: 0.8130 - val_loss: 0.3431 - val_acc: 0.8591
Epoch 354/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4598 - acc: 0.8155 - val_loss: 0.3434 - val_acc: 0.8644
Epoch 355/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4510 - acc: 0.8166 - val_loss: 0.3462 - val_acc: 0.8604
Epoch 356/500

Epoch 406/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4554 - acc: 0.8157 - val_loss: 0.3384 - val_acc: 0.8638
Epoch 407/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4467 - acc: 0.8171 - val_loss: 0.3412 - val_acc: 0.8618
Epoch 408/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4528 - acc: 0.8183 - val_loss: 0.3427 - val_acc: 0.8638
Epoch 409/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4481 - acc: 0.8183 - val_loss: 0.3380 - val_acc: 0.8651
Epoch 410/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4559 - acc: 0.8180 - val_loss: 0.3328 - val_acc: 0.8644
Epoch 411/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4481 - acc: 0.8188 - val_loss: 0.3375 - val_acc: 0.8638
Epoch 412/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4509 - acc: 0.8181 - val_loss: 0.3373 - val_acc: 0.8677
Epoch 413/500

Epoch 463/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4386 - acc: 0.8200 - val_loss: 0.3274 - val_acc: 0.8684
Epoch 464/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4461 - acc: 0.8192 - val_loss: 0.3277 - val_acc: 0.8644
Epoch 465/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4449 - acc: 0.8216 - val_loss: 0.3359 - val_acc: 0.8611
Epoch 466/500
15120/15120 [==============================] - 0s 6us/sample - loss: 0.4451 - acc: 0.8181 - val_loss: 0.3291 - val_acc: 0.8651
Epoch 467/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4474 - acc: 0.8202 - val_loss: 0.3303 - val_acc: 0.8684
Epoch 468/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4427 - acc: 0.8192 - val_loss: 0.3294 - val_acc: 0.8657
Epoch 469/500
15120/15120 [==============================] - 0s 7us/sample - loss: 0.4405 - acc: 0.8207 - val_loss: 0.3307 - val_acc: 0.8631
Epoch 470/500

In [31]:
y_pred = model.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8697089947089947
              precision    recall  f1-score   support

           1       0.75      0.77      0.76       196
           2       0.80      0.71      0.75       219
           3       0.88      0.75      0.81       228
           4       0.94      0.98      0.96       221
           5       0.94      0.98      0.96       201
           6       0.79      0.91      0.85       216
           7       0.98      0.98      0.98       231

    accuracy                           0.87      1512
   macro avg       0.87      0.87      0.87      1512
weighted avg       0.87      0.87      0.87      1512

[[151  38   1   0   1   1   4]
 [ 46 156   3   0  10   4   0]
 [  0   1 171  11   0  45   0]
 [  0   0   2 217   0   2   0]
 [  0   1   2   0 197   1   0]
 [  0   0  15   2   2 197   0]
 [  5   0   0   0   0   0 226]]


In [32]:
modelw = model2()
modelw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
modelw.fit(X, y, epochs=2000,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          class_weight = {0:3, 1:3, 2:1, 3:1, 4:1, 5:1, 6:1},
          batch_size=378)

Train on 15120 samples, validate on 1512 samples
Epoch 1/1000
15120/15120 [==============================] - 1s 48us/sample - loss: 2.1812 - acc: 0.2630 - val_loss: 1.3802 - val_acc: 0.4028
Epoch 2/1000
15120/15120 [==============================] - 0s 13us/sample - loss: 1.6971 - acc: 0.4101 - val_loss: 1.0024 - val_acc: 0.6138
Epoch 3/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 1.4402 - acc: 0.5098 - val_loss: 0.8454 - val_acc: 0.6587
Epoch 4/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.2970 - acc: 0.5636 - val_loss: 0.7897 - val_acc: 0.6726
Epoch 5/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.2350 - acc: 0.5860 - val_loss: 0.7671 - val_acc: 0.6878
Epoch 6/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 1.1843 - acc: 0.6054 - val_loss: 0.7299 - val_acc: 0.7083
Epoch 7/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 1.1384 - acc: 0.6232 - val_

Epoch 59/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8097 - acc: 0.7521 - val_loss: 0.4999 - val_acc: 0.8016
Epoch 60/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8188 - acc: 0.7507 - val_loss: 0.4959 - val_acc: 0.7976
Epoch 61/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8062 - acc: 0.7569 - val_loss: 0.4950 - val_acc: 0.7976
Epoch 62/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8091 - acc: 0.7476 - val_loss: 0.4922 - val_acc: 0.8056
Epoch 63/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8099 - acc: 0.7491 - val_loss: 0.4823 - val_acc: 0.8108
Epoch 64/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8097 - acc: 0.7534 - val_loss: 0.4904 - val_acc: 0.8095
Epoch 65/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.8014 - acc: 0.7524 - val_loss: 0.4897 - val_acc: 0.8042
Epoch 66/1000

Epoch 117/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7407 - acc: 0.7786 - val_loss: 0.4317 - val_acc: 0.8294
Epoch 118/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7321 - acc: 0.7796 - val_loss: 0.4244 - val_acc: 0.8366
Epoch 119/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7348 - acc: 0.7770 - val_loss: 0.4283 - val_acc: 0.8333
Epoch 120/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7276 - acc: 0.7802 - val_loss: 0.4285 - val_acc: 0.8247
Epoch 121/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7394 - acc: 0.7764 - val_loss: 0.4327 - val_acc: 0.8267
Epoch 122/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7335 - acc: 0.7769 - val_loss: 0.4385 - val_acc: 0.8201
Epoch 123/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7302 - acc: 0.7797 - val_loss: 0.4279 - val_acc: 0.8320
Epoch 

Epoch 175/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6887 - acc: 0.7964 - val_loss: 0.3900 - val_acc: 0.8512
Epoch 176/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7007 - acc: 0.7936 - val_loss: 0.3944 - val_acc: 0.8485
Epoch 177/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6914 - acc: 0.7911 - val_loss: 0.3972 - val_acc: 0.8485
Epoch 178/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.7034 - acc: 0.7894 - val_loss: 0.3965 - val_acc: 0.8433
Epoch 179/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6859 - acc: 0.7946 - val_loss: 0.3954 - val_acc: 0.8466
Epoch 180/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6980 - acc: 0.7927 - val_loss: 0.3949 - val_acc: 0.8452
Epoch 181/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6929 - acc: 0.7923 - val_loss: 0.3946 - val_acc: 0.8446
Epoch 

15120/15120 [==============================] - 0s 7us/sample - loss: 0.6689 - acc: 0.8040 - val_loss: 0.3839 - val_acc: 0.8552
Epoch 232/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6667 - acc: 0.8018 - val_loss: 0.3810 - val_acc: 0.8591
Epoch 233/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6675 - acc: 0.8054 - val_loss: 0.3787 - val_acc: 0.8598
Epoch 234/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6658 - acc: 0.8047 - val_loss: 0.3700 - val_acc: 0.8611
Epoch 235/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6731 - acc: 0.7987 - val_loss: 0.3679 - val_acc: 0.8611
Epoch 236/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6600 - acc: 0.8049 - val_loss: 0.3748 - val_acc: 0.8565
Epoch 237/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6683 - acc: 0.8005 - val_loss: 0.3672 - val_acc: 0.8604
Epoch 238/1000
15120/

...

15120/15120 [==============================] - 0s 7us/sample - loss: 0.6056 - acc: 0.8230 - val_loss: 0.3222 - val_acc: 0.8810
Epoch 737/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6107 - acc: 0.8198 - val_loss: 0.3219 - val_acc: 0.8843
Epoch 738/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6063 - acc: 0.8251 - val_loss: 0.3221 - val_acc: 0.8829
Epoch 739/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6117 - acc: 0.8251 - val_loss: 0.3228 - val_acc: 0.8810
Epoch 740/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6085 - acc: 0.8224 - val_loss: 0.3226 - val_acc: 0.8849
Epoch 741/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6064 - acc: 0.8225 - val_loss: 0.3212 - val_acc: 0.8836
Epoch 742/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6022 - acc: 0.8253 - val_loss: 0.3216 - val_acc: 0.8856
Epoch 743/1000
15120/

15120/15120 [==============================] - 0s 7us/sample - loss: 0.6132 - acc: 0.8185 - val_loss: 0.3195 - val_acc: 0.8829
Epoch 793/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6030 - acc: 0.8235 - val_loss: 0.3195 - val_acc: 0.8862
Epoch 794/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6067 - acc: 0.8240 - val_loss: 0.3208 - val_acc: 0.8829
Epoch 795/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6059 - acc: 0.8244 - val_loss: 0.3201 - val_acc: 0.8829
Epoch 796/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6058 - acc: 0.8218 - val_loss: 0.3205 - val_acc: 0.8803
Epoch 797/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6090 - acc: 0.8227 - val_loss: 0.3207 - val_acc: 0.8829
Epoch 798/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6138 - acc: 0.8219 - val_loss: 0.3207 - val_acc: 0.8849
Epoch 799/1000
15120/

 7938/15120 [==============>...............] - ETA: 0s - loss: 0.5900 - acc: 0.8318
Epoch 00848: ReduceLROnPlateau reducing learning rate to 4.398046876303852e-05.
15120/15120 [==============================] - 0s 8us/sample - loss: 0.5992 - acc: 0.8296 - val_loss: 0.3198 - val_acc: 0.8836
Epoch 849/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6023 - acc: 0.8269 - val_loss: 0.3191 - val_acc: 0.8856
Epoch 850/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6007 - acc: 0.8218 - val_loss: 0.3193 - val_acc: 0.8869
Epoch 851/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6051 - acc: 0.8221 - val_loss: 0.3193 - val_acc: 0.8869
Epoch 852/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6039 - acc: 0.8255 - val_loss: 0.3193 - val_acc: 0.8862
Epoch 853/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6086 - acc: 0.8262 - val_loss: 0.3194 - val_acc: 0.8829

15120/15120 [==============================] - 0s 7us/sample - loss: 0.6009 - acc: 0.8290 - val_loss: 0.3187 - val_acc: 0.8843
Epoch 904/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6071 - acc: 0.8214 - val_loss: 0.3192 - val_acc: 0.8816
Epoch 905/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6052 - acc: 0.8258 - val_loss: 0.3189 - val_acc: 0.8829
Epoch 906/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6043 - acc: 0.8253 - val_loss: 0.3192 - val_acc: 0.8829
Epoch 907/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6097 - acc: 0.8233 - val_loss: 0.3192 - val_acc: 0.8823
Epoch 908/1000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.6057 - acc: 0.8236 - val_loss: 0.3193 - val_acc: 0.8836
Epoch 909/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6047 - acc: 0.8285 - val_loss: 0.3195 - val_acc: 0.8862
Epoch 910/1000
15120/

15120/15120 [==============================] - 0s 7us/sample - loss: 0.5975 - acc: 0.8283 - val_loss: 0.3188 - val_acc: 0.8836
Epoch 960/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5952 - acc: 0.8272 - val_loss: 0.3184 - val_acc: 0.8849
Epoch 961/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5995 - acc: 0.8276 - val_loss: 0.3187 - val_acc: 0.8856
Epoch 962/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6100 - acc: 0.8241 - val_loss: 0.3187 - val_acc: 0.8843
Epoch 963/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6039 - acc: 0.8248 - val_loss: 0.3184 - val_acc: 0.8856
Epoch 964/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.6125 - acc: 0.8206 - val_loss: 0.3191 - val_acc: 0.8836
Epoch 965/1000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.5998 - acc: 0.8255 - val_loss: 0.3185 - val_acc: 0.8843
Epoch 966/1000
15120/

In [33]:
y_pred = modelw.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8855820105820106
              precision    recall  f1-score   support

           1       0.75      0.85      0.80       196
           2       0.84      0.75      0.80       219
           3       0.91      0.77      0.83       228
           4       0.95      0.99      0.97       221
           5       0.95      0.96      0.95       201
           6       0.81      0.93      0.87       216
           7       0.99      0.95      0.97       231

    accuracy                           0.89      1512
   macro avg       0.89      0.89      0.88      1512
weighted avg       0.89      0.89      0.89      1512

[[167  25   0   0   1   1   2]
 [ 43 165   1   0   8   2   0]
 [  0   2 175  10   0  41   0]
 [  0   0   1 219   0   1   0]
 [  0   3   4   0 193   1   0]
 [  0   1  11   1   2 201   0]
 [ 12   0   0   0   0   0 219]]


In [48]:
# modelw = model2()
# modelw.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])
modelw.fit(X, y, epochs=5000,
          callbacks=[early_stop, reduce_lr, mcp_save, tensorboard_logs],
          validation_data=(X_val, y_val),
          class_weight = {0:4, 1:4, 2:1, 3:1, 4:1, 5:2, 6:1},
          batch_size=378)

Train on 15120 samples, validate on 1512 samples
Epoch 1/5000
15120/15120 [==============================] - 1s 34us/sample - loss: 0.9611 - acc: 0.8182 - val_loss: 0.3485 - val_acc: 0.8644
Epoch 2/5000
15120/15120 [==============================] - 0s 15us/sample - loss: 0.9694 - acc: 0.8152 - val_loss: 0.3485 - val_acc: 0.8644
Epoch 3/5000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.9749 - acc: 0.8130 - val_loss: 0.3486 - val_acc: 0.8644
Epoch 4/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9680 - acc: 0.8118 - val_loss: 0.3486 - val_acc: 0.8644
Epoch 5/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9903 - acc: 0.8090 - val_loss: 0.3485 - val_acc: 0.8644
Epoch 6/5000
15120/15120 [==============================] - 0s 8us/sample - loss: 0.9599 - acc: 0.8150 - val_loss: 0.3485 - val_acc: 0.8644
Epoch 7/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9563 - acc: 0.8151 - val_

15120/15120 [==============================] - 0s 7us/sample - loss: 0.9724 - acc: 0.8155 - val_loss: 0.3491 - val_acc: 0.8631
Epoch 58/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9782 - acc: 0.8153 - val_loss: 0.3491 - val_acc: 0.8631
Epoch 59/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9746 - acc: 0.8140 - val_loss: 0.3491 - val_acc: 0.8631
Epoch 60/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9695 - acc: 0.8163 - val_loss: 0.3492 - val_acc: 0.8631
Epoch 61/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9753 - acc: 0.8136 - val_loss: 0.3492 - val_acc: 0.8631
Epoch 62/5000
15120/15120 [==============================] - 0s 7us/sample - loss: 0.9804 - acc: 0.8132 - val_loss: 0.3491 - val_acc: 0.8631


In [46]:
y_pred = modelw.predict(X_val)
report(binarizer.inverse_transform(y_val), binarizer.inverse_transform(y_pred))

Accuracy: 0.8624338624338624
              precision    recall  f1-score   support

           1       0.73      0.85      0.79       196
           2       0.81      0.78      0.80       219
           3       0.94      0.64      0.76       228
           4       0.95      0.98      0.97       221
           5       0.97      0.93      0.95       201
           6       0.73      0.95      0.82       216
           7       0.99      0.91      0.95       231

    accuracy                           0.86      1512
   macro avg       0.87      0.86      0.86      1512
weighted avg       0.88      0.86      0.86      1512

[[167  26   0   0   0   1   2]
 [ 41 171   0   0   6   1   0]
 [  0   2 145  10   0  71   0]
 [  0   0   0 217   0   4   0]
 [  0   9   4   0 187   1   0]
 [  1   2   6   1   0 206   0]
 [ 19   1   0   0   0   0 211]]


## Predictions

In [34]:
test_pred = modelw.predict(X_test)

In [35]:
# Save test predictions to file
output = pd.DataFrame({'ID': test_ids,
                       'Cover_Type': binarizer.inverse_transform(test_pred)})
output.to_csv('submission_neural.csv', index=False)